In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import os
from os import path
import sys

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
sys.path.append('/mnt/c/Users/44yos/RacePrediction/resnet_win5')

from pickle import load

# from models import resnet
from models import transformer

from tensorflow.python import keras
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras import optimizers
from tensorflow.python.keras import backend as K


# from utils import functions
from training.utils import create_time_series_data_weekly, smooth_label, categorical_focal_loss, order_algorithm

In [2]:
data = pd.read_csv("../data/csv/2022/Oct_2/10_16_data.csv", sep = ",")

In [3]:
data['race_id'] = data['race_id'].astype(str)
data['race_round'] = data['race_round'].astype(str)
data['order'] = data['order'].astype(str)
data['frame_number'] = data['frame_number'].astype(str)
data['horse_number'] = data['horse_number'].astype(str)

In [4]:
print(data.isnull().sum())

race_id                       0
race_round                    0
race_title                    0
ground_condition              0
date                          0
total_horse_number            0
order                         0
frame_number                  0
horse_number                  0
horse_id                      0
age                           0
burden_weight                 0
goal_time                     0
half_order                    0
last_time                     0
odds                          0
horse_weight                  0
race_rank                     0
distance                      0
ground_type_ダ                 0
ground_type_芝                 0
circle_右                      0
circle_左                      0
circle_右.1                    0
circle_左.1                    0
weather_circumstance_晴        0
weather_circumstance_曇        0
weather_circumstance_小雨       0
weather_circumstance_雨        0
weather_circumstance_晴.1      0
weather_circumstance_曇.1      0
weather_

In [5]:
data = data.fillna(data.mean())

In [6]:
labels = data[["race_id","race_title","odds", "order", "horse_number", "goal_time", "half_order", "last_time", "horse_weight"]]

In [7]:
data = data.drop(["horse_number", "half_order", "goal_time" ,"last_time", "horse_weight",  "frame_number"], axis = 1)
print(data.shape)#"horse_weight_dif",,"pop"


(161, 150)


In [8]:
# 列の順序変更  "東京":'place_東京',"阪神":'place_阪神',"新潟":'place_新潟',
data = data.rename(columns ={"中京":'place_中京',"中山":'place_中山',"札幌":'place_札幌',"小倉":'place_小倉',"福島":'place_福島',"函館":'place_函館',"京都":'place_京都'})
data = data[['race_id','order',"race_round","ground_condition","total_horse_number","age","burden_weight","odds","race_rank","distance","ground_type_ダ","ground_type_芝","circle_右","circle_左","weather_circumstance_小雨","weather_circumstance_小雪","weather_circumstance_晴","weather_circumstance_曇","weather_circumstance_雨","weather_circumstance_雪","place_中京","place_中山","place_京都","place_函館","place_小倉","place_新潟","place_札幌","place_東京","place_福島","place_阪神","sex_セ","sex_牝","sex_牡","f_grass_win_rate","f_dart_win_rate","f_win_rate","g_f_grass_win_rate","g_f_dart_win_rate","g_f_win_rate","m_grass_win_rate","m_dart_win_rate","m_win_rate","whole_horse_number_1","odds_1","order_1","burden_weight_1","race_distance_1","ground_condition_1","goal_time_1","half_order_1","last_time_1","horse_weight_1","weather_circumstance_小雨_1","weather_circumstance_小雪_1","weather_circumstance_晴_1","weather_circumstance_曇_1","weather_circumstance_雨_1","weather_circumstance_雪_1","main_place_その他_1","main_place_中京_1","main_place_中山_1","main_place_京都_1","main_place_函館_1","main_place_小倉_1","main_place_新潟_1","main_place_札幌_1","main_place_東京_1","main_place_福島_1","main_place_阪神_1","race_rank_1","ground_type_ダ_1","ground_type_芝_1","ground_type_障_1","horse_weight_dif_1","same_jockey_1","whole_horse_number_2","odds_2","order_2","burden_weight_2","race_distance_2","ground_condition_2","goal_time_2","half_order_2","last_time_2","horse_weight_2","weather_circumstance_小雨_2","weather_circumstance_小雪_2","weather_circumstance_晴_2","weather_circumstance_曇_2","weather_circumstance_雨_2","weather_circumstance_雪_2","main_place_その他_2","main_place_中京_2","main_place_中山_2","main_place_京都_2","main_place_函館_2","main_place_小倉_2","main_place_新潟_2","main_place_札幌_2","main_place_東京_2","main_place_福島_2","main_place_阪神_2","race_rank_2","ground_type_ダ_2","ground_type_芝_2","ground_type_障_2","horse_weight_dif_2","same_jockey_2","whole_horse_number_3","odds_3","order_3","burden_weight_3","race_distance_3","ground_condition_3","goal_time_3","half_order_3","last_time_3","horse_weight_3","weather_circumstance_小雨_3","weather_circumstance_小雪_3","weather_circumstance_晴_3","weather_circumstance_曇_3","weather_circumstance_雨_3","weather_circumstance_雪_3","main_place_その他_3","main_place_中京_3","main_place_中山_3","main_place_京都_3","main_place_函館_3","main_place_小倉_3","main_place_新潟_3","main_place_札幌_3","main_place_東京_3","main_place_福島_3" ,"main_place_阪神_3","race_rank_3","ground_type_ダ_3","ground_type_芝_3","ground_type_障_3","horse_weight_dif_3","same_jockey_3","same_jockey"]]
print(data.shape)

(161, 142)


In [9]:
print(data.dtypes)

race_id                       object
order                         object
race_round                    object
ground_condition               int64
total_horse_number             int64
age                            int64
burden_weight                float64
odds                         float64
race_rank                      int64
distance                       int64
ground_type_ダ                  int64
ground_type_芝                  int64
circle_右                       int64
circle_左                       int64
weather_circumstance_小雨        int64
weather_circumstance_小雪        int64
weather_circumstance_晴         int64
weather_circumstance_曇         int64
weather_circumstance_雨         int64
weather_circumstance_雪         int64
place_中京                       int64
place_中山                       int64
place_京都                       int64
place_函館                       int64
place_小倉                       int64
place_新潟                       int64
place_札幌                       int64
p

In [10]:
# standard_scale = StandardScaler()
no_scale_data = data[['race_id','order']]
scale_columns = data.drop(['race_id','order'], axis=1).columns.values
standard_scale = load(open("../training/standard_scale.pkl", "rb"))
data = pd.DataFrame(standard_scale.transform(data[scale_columns]))

# PCA
pca = load(open("../training/pca.pkl", "rb"))
data = pd.DataFrame(pca.transform(data))
max_col = 84
# print(max_col)
print(data.shape)
data = data.loc[:, :max_col-1]

data = pd.concat([data, no_scale_data], axis=1)
print(data.shape)

(161, 140)
(161, 86)


In [11]:
# def create_time_series_data_weekly(raw_data):
#     number_of_race = raw_data.race_id.nunique()
#     time_series_data = np.full((number_of_race, 18, 140), 0.0)#-float('inf')
#     race_number = 0
#     horse_number = 0
#     for i in range(len(raw_data)):
#         if i == 0:
#             time_series_data[race_number][horse_number] = raw_data.iloc[i].drop(['race_id','order'])
#             horse_number += 1
#             continue
#         if data.iloc[i].race_id != data.iloc[i-1].race_id:
#             race_number += 1
#             horse_number = 0
#             time_series_data[race_number][horse_number] = raw_data.iloc[i].drop(['race_id','order'])
#             horse_number += 1
#         else:
#             time_series_data[race_number][horse_number] = raw_data.iloc[i].drop(['race_id','order'])
#             horse_number += 1
#     return time_series_data

In [12]:
X = create_time_series_data_weekly(data)
print(X.shape)

(12, 24, 84)


In [13]:
print(X[1])

[[ 1.0271445  -0.94551508  0.50530929 ...  0.75013012 -0.20777028
  -0.2935086 ]
 [ 0.80654308  1.45146636 -0.49109289 ... -0.62882672 -0.19144443
  -0.17923016]
 [ 3.66191459 -3.8104794   3.89300896 ...  1.09041133  0.0268159
   0.17089589]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [14]:
X = X.astype('float32')
print(X.dtype)

float32


In [15]:
# def categorical_focal_loss(alpha, gamma):
#     """
#     Softmax version of focal loss.
#     When there is a skew between different categories/labels in your data set, you can try to apply this function as a
#     loss.
#            m
#       FL = ∑  -alpha * (1 - p_o,c)^gamma * y_o,c * log(p_o,c)
#           c=1
#       where m = number of classes, c = class and o = observation
#     Parameters:
#       alpha -- the same as weighing factor in balanced cross entropy. Alpha is used to specify the weight of different
#       categories/labels, the size of the array needs to be consistent with the number of classes.
#       gamma -- focusing parameter for modulating factor (1-p)
#     Default value:
#       gamma -- 2.0 as mentioned in the paper
#       alpha -- 0.25 as mentioned in the paper
#     References:
#         Official paper: https://arxiv.org/pdf/1708.02002.pdf
#         https://www.tensorflow.org/api_docs/python/tf/keras/backend/categorical_crossentropy
#     Usage:
#      model.compile(loss=[categorical_focal_loss(alpha=[[.25, .25, .25]], gamma=2)], metrics=["accuracy"], optimizer=adam)
#     """

#     alpha = np.array(alpha, dtype=np.float32)

#     def categorical_focal_loss_fixed(y_true, y_pred):
#         """
#         :param y_true: A tensor of the same shape as `y_pred`
#         :param y_pred: A tensor resulting from a softmax
#         :return: Output tensor.
#         """

#         # Clip the prediction value to prevent NaN's and Inf's
#         epsilon = K.epsilon()
#         y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

#         # Calculate Cross Entropy
#         cross_entropy = -y_true * K.log(y_pred)

#         # Calculate Focal Loss
#         loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

#         # Compute mean loss in mini_batch
#         return K.mean(K.sum(loss, axis=-1))

#     return categorical_focal_loss_fixed

In [16]:
alpha = [0.00000000e+00, 1.21687621e+00, 1.21555368e+00, 1.23063757e+00,
         1.24041898e+00, 1.25990362e+00, 1.26100846e+00, 1.28155834e+00,
         1.32325490e+00, 1.37157651e+00, 1.45736210e+00, 1.56413643e+00,
         1.73521807e+00, 1.98052140e+00, 2.27608819e+00, 2.71833648e+00,
         3.58156912e+00, 1.49347181e+01, 2.00918164e+01, 1.25825000e+03,
         1.43800000e+03, 2.51650000e+03, 3.35533333e+03, 4.02640000e+03,
         5.03300000e+03, 7.53440294e-02]
# [0., 1.22620251, 1.21236789, 1.23692263, 1.24838235, 1.27347735, 1.27233213, 1.2771175, 1.33202573, 1.35824, 1.42768248, 1.52624955, 1.69170984, 1.90080609, 2.16445691, 2.54619076, 3.30697312, 14.51111111, 19.42562929, 0.13969294]

In [17]:
num_layers = 2 # hyperparameter
d_model = max_col # 4*35 84=4*3*7
num_heads = 28 # hyperparameter *must be a factor of d_model*
d_ffn = 16 # hyperparameter
pe_input = 24
target_size = 26
dropout_rate = 0.1 # hyperparameter

model1 = transformer.TransRace(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    d_ffn=d_ffn,
    pe_input=pe_input,
    target_size=target_size,
    rate=dropout_rate,
                              )
opt = optimizers.Adam(decay=0.01)
loss = categorical_focal_loss(alpha=[alpha], gamma=0.5)
model1.compile(
    optimizer=opt,
    loss=loss,#'categorical_crossentropy', # 'sigmoid_focal_crossentropy'
    metrics=['accuracy'], #['categorical_accuracy']
)
model1.build(input_shape=(None, X.shape[1], X.shape[2]))
model1.load_weights("../models/results/transformer1.h5")
pred1 = model1.predict(X)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/home/yoshi/miniconda3/envs/tensorflow-gpu/lib/python3.6/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


    def call(self, x, training, mask):
#         print(x.shape,"mha")
        attn_output = self.mha(x, x, x, mask=mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

This may be caused by multiline strings or comments not indented at the same level as the code.
    def call(self, x, training, mask):
#         print(x.shape,"mha")
        attn_output = self.mha(x, x, x, mask=mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_ou

    def call(self, x, training, mask):
#         print(x.shape,"mha")
        attn_output = self.mha(x, x, x, mask=mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

This may be caused by multiline strings or comments not indented at the same level as the code.
    def call(self, x, training, mask):
#         print(x.shape,"mha")
        attn_output = self.mha(x, x, x, mask=mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_ou

    def call(self, inputs, training=True):
        inp = inputs

        enc_padding_mask = self.create_masks(inp) #, tar
        # , look_ahead_mask, dec_padding_mask

        enc_output = self.encoder(inp, training, enc_padding_mask)

        final_output = self.final_layer(enc_output)
#         print("final output shape", final_output.shape)

        return final_output

This may be caused by multiline strings or comments not indented at the same level as the code.
    def call(self, inputs, training=True):
        inp = inputs

        enc_padding_mask = self.create_masks(inp) #, tar
        # , look_ahead_mask, dec_padding_mask

        enc_output = self.encoder(inp, training, enc_padding_mask)

        final_output = self.final_layer(enc_output)
#         print("final output shape", final_output.shape)

        return final_output

This may be caused by multiline strings or comments not indented at the same level as the code.
    def call(self, x, training, mask):
#         print

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, self.seq_len, self.num_heads, self.depth))
#         print(x.shape,"split_head")
        return tf.transpose(x, perm=[0, 2, 1, 3])

This may be caused by multiline strings or comments not indented at the same level as the code.
    def call(self, x, training, mask):
#         print(x.shape,"mha")
        attn_output = self.mha(x, x, x, mask=mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, 

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, self.seq_len, self.num_heads, self.depth))
#         print(x.shape,"split_head")
        return tf.transpose(x, perm=[0, 2, 1, 3])

This may be caused by multiline strings or comments not indented at the same level as the code.


In [18]:
num_layers = 2 # hyperparameter
d_model = max_col # 4*35
num_heads = 21 # hyperparameter *must be a factor of d_model*
d_ffn = 16 # hyperparameter
pe_input = 24
target_size = 26
dropout_rate = 0.1 # hyperparameter

model2 = transformer.TransRace(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    d_ffn=d_ffn,
    pe_input=pe_input,
    target_size=target_size,
    rate=dropout_rate,
                              )
opt = optimizers.Adam(decay=0.01)
loss = categorical_focal_loss(alpha=[alpha], gamma=0.5)
model2.compile(
    optimizer=opt,
    loss=loss,#'categorical_crossentropy', # 'sigmoid_focal_crossentropy'
    metrics=['accuracy'], #['categorical_accuracy']
)
model2.build(input_shape=(None, X.shape[1], X.shape[2]))
model2.load_weights("../models/results/transformer2.h5")
pred2 = model2.predict(X)

    def call(self, x, training, mask):
#         print(x.shape,"mha")
        attn_output = self.mha(x, x, x, mask=mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

This may be caused by multiline strings or comments not indented at the same level as the code.
    def call(self, x, training, mask):
#         print(x.shape,"mha")
        attn_output = self.mha(x, x, x, mask=mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_ou

    def call(self, x, training, mask):
#         print(x.shape,"mha")
        attn_output = self.mha(x, x, x, mask=mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

This may be caused by multiline strings or comments not indented at the same level as the code.
    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, self.seq_len, self.num_heads, self.depth))
#         print(x.shape,"split_head")
     

    def call(self, x, training, mask):
#         print(x.shape,"mha")
        attn_output = self.mha(x, x, x, mask=mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

This may be caused by multiline strings or comments not indented at the same level as the code.
    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, self.seq_len, self.num_heads, self.depth))
#         print(x.shape,"split_head")
     

    def call(self, x, training, mask):
#         print(x.shape,"mha")
        attn_output = self.mha(x, x, x, mask=mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

This may be caused by multiline strings or comments not indented at the same level as the code.
    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, self.seq_len, self.num_heads, self.depth))
#         print(x.shape,"split_head")
     

In [19]:
num_layers = 1 # hyperparameter
d_model = max_col # 4*35
num_heads = 21 # hyperparameter *must be a factor of d_model*
d_ffn = 64 # hyperparameter
pe_input = 24
target_size = 26
dropout_rate = 0.1 # hyperparameter

model3 = transformer.TransRace(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    d_ffn=d_ffn,
    pe_input=pe_input,
    target_size=target_size,
    rate=dropout_rate,
                              )
opt = optimizers.Adam(decay=0.01)
loss = categorical_focal_loss(alpha=[alpha], gamma=0.5)
model3.compile(
    optimizer=opt,
    loss=loss,#'categorical_crossentropy', # 'sigmoid_focal_crossentropy'
    metrics=['accuracy'], #['categorical_accuracy']
)
model3.build(input_shape=(None, X.shape[1], X.shape[2]))
model3.load_weights("../models/results/transformer3.h5")
pred3 = model3.predict(X)

    def call(self, x, training, mask):
#         print(x.shape,"mha")
        attn_output = self.mha(x, x, x, mask=mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

This may be caused by multiline strings or comments not indented at the same level as the code.
    def call(self, x, training, mask):
#         print(x.shape,"mha")
        attn_output = self.mha(x, x, x, mask=mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_ou

    def call(self, inputs, training=True):
        inp = inputs

        enc_padding_mask = self.create_masks(inp) #, tar
        # , look_ahead_mask, dec_padding_mask

        enc_output = self.encoder(inp, training, enc_padding_mask)

        final_output = self.final_layer(enc_output)
#         print("final output shape", final_output.shape)

        return final_output

This may be caused by multiline strings or comments not indented at the same level as the code.
    def call(self, inputs, training=True):
        inp = inputs

        enc_padding_mask = self.create_masks(inp) #, tar
        # , look_ahead_mask, dec_padding_mask

        enc_output = self.encoder(inp, training, enc_padding_mask)

        final_output = self.final_layer(enc_output)
#         print("final output shape", final_output.shape)

        return final_output

This may be caused by multiline strings or comments not indented at the same level as the code.
    def call(self, x, training, mask):
#         print

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, self.seq_len, self.num_heads, self.depth))
#         print(x.shape,"split_head")
        return tf.transpose(x, perm=[0, 2, 1, 3])

This may be caused by multiline strings or comments not indented at the same level as the code.


In [20]:
num_layers = 1 # hyperparameter
d_model = max_col # 4*35
num_heads = 21 # hyperparameter *must be a factor of d_model*
d_ffn = 64 # hyperparameter
pe_input = 24
target_size = 26
dropout_rate = 0.1 # hyperparameter

model4 = transformer.TransRace(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    d_ffn=d_ffn,
    pe_input=pe_input,
    target_size=target_size,
    rate=dropout_rate,
                              )
opt = optimizers.Adam(decay=0.01)
loss = categorical_focal_loss(alpha=[alpha], gamma=0.5)
model4.compile(
    optimizer=opt,
    loss=loss,#'categorical_crossentropy', # 'sigmoid_focal_crossentropy'
    metrics=['accuracy'], #['categorical_accuracy']
)
model4.build(input_shape=(None, X.shape[1], X.shape[2]))
model4.load_weights("../models/results/transformer4.h5")
pred4 = model4.predict(X)

    def call(self, x, training, mask):
#         print(x.shape,"mha")
        attn_output = self.mha(x, x, x, mask=mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

This may be caused by multiline strings or comments not indented at the same level as the code.
    def call(self, x, training, mask):
#         print(x.shape,"mha")
        attn_output = self.mha(x, x, x, mask=mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_ou

    def call(self, inputs, training=True):
        inp = inputs

        enc_padding_mask = self.create_masks(inp) #, tar
        # , look_ahead_mask, dec_padding_mask

        enc_output = self.encoder(inp, training, enc_padding_mask)

        final_output = self.final_layer(enc_output)
#         print("final output shape", final_output.shape)

        return final_output

This may be caused by multiline strings or comments not indented at the same level as the code.
    def call(self, inputs, training=True):
        inp = inputs

        enc_padding_mask = self.create_masks(inp) #, tar
        # , look_ahead_mask, dec_padding_mask

        enc_output = self.encoder(inp, training, enc_padding_mask)

        final_output = self.final_layer(enc_output)
#         print("final output shape", final_output.shape)

        return final_output

This may be caused by multiline strings or comments not indented at the same level as the code.
    def call(self, x, training, mask):
#         print

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, self.seq_len, self.num_heads, self.depth))
#         print(x.shape,"split_head")
        return tf.transpose(x, perm=[0, 2, 1, 3])

This may be caused by multiline strings or comments not indented at the same level as the code.


In [21]:
num_layers = 1 # hyperparameter
d_model = max_col # 4*35
num_heads = 21 # hyperparameter *must be a factor of d_model*
d_ffn = 128 # hyperparameter
pe_input = 24
target_size = 26
dropout_rate = 0.1 # hyperparameter

model5 = transformer.TransRace(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    d_ffn=d_ffn,
    pe_input=pe_input,
    target_size=target_size,
    rate=dropout_rate,
                              )
opt = optimizers.Adam(decay=0.01)
loss = categorical_focal_loss(alpha=[alpha], gamma=0.5)
model5.compile(
    optimizer=opt,
    loss=loss,#'categorical_crossentropy', # 'sigmoid_focal_crossentropy'
    metrics=['accuracy'], #['categorical_accuracy']
)
model5.build(input_shape=(None, X.shape[1], X.shape[2]))
model5.load_weights("../models/results/transformer5.h5")
pred5 = model5.predict(X)

    def call(self, x, training, mask):
#         print(x.shape,"mha")
        attn_output = self.mha(x, x, x, mask=mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

This may be caused by multiline strings or comments not indented at the same level as the code.
    def call(self, x, training, mask):
#         print(x.shape,"mha")
        attn_output = self.mha(x, x, x, mask=mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_ou

    def call(self, inputs, training=True):
        inp = inputs

        enc_padding_mask = self.create_masks(inp) #, tar
        # , look_ahead_mask, dec_padding_mask

        enc_output = self.encoder(inp, training, enc_padding_mask)

        final_output = self.final_layer(enc_output)
#         print("final output shape", final_output.shape)

        return final_output

This may be caused by multiline strings or comments not indented at the same level as the code.
    def call(self, inputs, training=True):
        inp = inputs

        enc_padding_mask = self.create_masks(inp) #, tar
        # , look_ahead_mask, dec_padding_mask

        enc_output = self.encoder(inp, training, enc_padding_mask)

        final_output = self.final_layer(enc_output)
#         print("final output shape", final_output.shape)

        return final_output

This may be caused by multiline strings or comments not indented at the same level as the code.
    def call(self, x, training, mask):
#         print

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, self.seq_len, self.num_heads, self.depth))
#         print(x.shape,"split_head")
        return tf.transpose(x, perm=[0, 2, 1, 3])

This may be caused by multiline strings or comments not indented at the same level as the code.


In [22]:
# log_pred1 = np.log(pred1)
# log_pred2 = np.log(pred2)
# log_pred3 = np.log(pred3)
# log_pred4 = np.log(pred4)
# log_pred5 = np.log(pred5)
# log_pred6 = np.log(pred6)
# log_pred7 = np.log(pred7)
# log_pred8 = np.log(pred8)
# log_pred9 = np.log(pred9)
# log_pred10 = np.log(pred10)

sum_pred = (pred1 + pred2 + pred3 + pred4 + pred5) * 100 / 5 #log_pred1 + log_pred2 + log_pred3 + log_pred4 + log_pred5 # + log_pred6 + log_pred7 + log_pred8 + log_pred9 + log_pred10
pred_order = (order_algorithm(sum_pred)).flatten()
sum_pred = sum_pred.reshape([-1, 26])
print(sum_pred.shape)
# pred_order = np.argmax(sum_pred, axis = -1)
print(pred_order.shape)

(288, 26)
(288,)


In [23]:
pred_df = pd.DataFrame(sum_pred, columns = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25'])
pred_order_df = pd.DataFrame(pred_order, columns = ["pred"])
summary = labels
summary = pd.concat([summary,pred_order_df], axis = 1)
summary = pd.concat([summary,pred_df], axis = 1)

In [24]:
print(summary.head(26))
# print(pred_order_df.head(96))

            race_id race_title   odds order horse_number goal_time half_order  \
0   202204040208&rf  3歳以上1勝クラス    9.6     ?            1         ?          ?   
1   202204040208&rf  3歳以上1勝クラス   24.4     ?            2         ?          ?   
2   202204040208&rf  3歳以上1勝クラス   48.9     ?            3         ?          ?   
3   202204040208&rf  3歳以上1勝クラス   42.7     ?            4         ?          ?   
4   202204040208&rf  3歳以上1勝クラス   90.1     ?            5         ?          ?   
5   202204040208&rf  3歳以上1勝クラス    3.8     ?            6         ?          ?   
6   202204040208&rf  3歳以上1勝クラス   10.9     ?            7         ?          ?   
7   202204040208&rf  3歳以上1勝クラス    3.6     ?            8         ?          ?   
8   202204040208&rf  3歳以上1勝クラス   16.3     ?            9         ?          ?   
9   202204040208&rf  3歳以上1勝クラス    8.8     ?           10         ?          ?   
10  202204040208&rf  3歳以上1勝クラス   45.9     ?           11         ?          ?   
11  202204040208&rf  3歳以上1勝ク

In [25]:
os.makedirs('../../weekly_prediction/2022/Oct_2', exist_ok = True)
summary[["race_title","odds","horse_number","pred",'1','2','3']].to_csv("../../weekly_prediction/2022/Oct_2/10_16_race_pred.csv", index = False)

In [ ]:
# summary[["pred",'1','2','3']].to_csv("csv/data/jan_1/1_24_race_pred_v2.csv", index = False)

In [ ]:
# summary[["pred",'1','2','3']].to_csv("csv/data/jan_1/1_24_race_pred_v2.csv", index = False)

In [ ]:
# summary[["pred",'1','2','3']].to_csv("csv/data/jan_1/1_24_race_pred_v2.csv", index = False)

In [ ]:
# summary[["pred",'1','2','3']].to_csv("csv/data/jan_1/1_24_race_pred_v2.csv", index = False)